# Heatmap with all QBs and their EPA per pass type

In [2]:
#Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Import the data for the whole 2020 season
YEAR = 2020
data = pd.read_csv(
    'https://github.com/guga31bb/nflfastR-data/blob/master/data/play_by_play_' \
    + str(YEAR) + '.csv.gz?raw=True',compression='gzip', low_memory=False
   )

In [4]:
#Add a new column 'pass type' by concatenating pass length and pass location
data['pass_type'] = data['pass_length'] + ' - ' + data['pass_location']

In [8]:
#Indicate the most recent week, or the one you want to analyze in particular
week = 12

#Clean the data and only keep the pass plays. Note: test if the other conditions could be removed
df = data[
    (data.week<=week) &
    (data.play_type.isin(['pass'])) &
    (data.sack==0) &      
    (data.two_point_attempt==0) &
    (data['passer_player_name'].isna()==False)
    ]

In [6]:
#Correcting G.Minshew II that is present twice
df['passer_player_name'] = df['passer_player_name'].replace(['G.Minshew II'],'G.Minshew')

C:\Users\nicol\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
#Make an Excel-like pivot table and calculate the average WPA per pass type
table = pd.pivot_table(df, values='epa', index=['passer_player_name'],columns=['pass_type'], aggfunc=np.mean)

#Transpose it in order to have the QBs as columns
grouped = table.transpose()

#Pass the names off QBs in a list
qb_list = grouped.columns.tolist()

In [10]:
#Go through each QB of the list, take his wpa, pass it into a heatmap and save as a png
#Close the plt each time, otherwise the viz will merge into one unreadable monster
for qb in qb_list:
    epa = grouped[qb].reset_index()
    plot_data = (epa.join(epa['pass_type'].str.split(' - ', expand=True))
               .pivot(index=0, columns=1, values=qb)
            )
    plot_data.index.name = "Pass length"
    plot_data.columns.name = "Pass location"
    ax = sns.heatmap(plot_data, annot=True, linewidths=.4,cmap="RdYlGn", vmin=-3,vmax=3)
    plt.title(f'{qb} average EPA')
    plt.savefig(f'{qb}.png')
    plt.close()